In [2]:
import requests
from bs4 import BeautifulSoup
import ollama
import json

In [3]:
url = "https://www.nationalgeographic.com.es/ciencia/porfiria-enfermedad-metabolica-dio-lugar-vampiros_26542"

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
response = requests.get(url, headers=headers)
    

soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
title = soup.find("h1", class_="article-title").text.strip()

In [6]:
print(title)

Porfiria, la enfermedad metabólica que dio lugar a los vampiros


In [7]:
paragraphs = soup.find_all('div', class_='article-text txt')

In [8]:
text = ""
for i in paragraphs[0].find_all("p"):
    if not i.attrs:
        print(i.text)
        text += i.text + "\n"

A lo largo de la historia de la humanidad, los vampiros han sido objeto de fascinación, de miedo y mitos. Hoy en día, la leyenda sobre la crueldad del Conde Drácula, así como otras criaturas nocturnas han hecho mella en el imaginario popular, aunque sus relatos se han ido suavizando desde finales del siglo XIX hasta la fecha. Con el paso de los años, las habilidades de los vampiros han ido variando, pero por lo general siempre han sido criaturas esquivas, a los que no les puede dar el sol y a los que se le combate con plata, ajos, y, para darles la estocada final, se necesita una estaca de madera.
Si me invitan a pasar a través de sus pantallas, podrán leer la historia de cómo las leyendas se apoyan en fenómenos otrora inexplicables, pero que en la actualidad su halo de misticismo ha ido desvaneciéndose gracias a la ciencia. En el caso de los vampiros, estos seres podrían haber surgido a partir de enfermedades metabólicas raras llamadas porfirias, cuyos síntomas se asemejan vagamente a

In [9]:
prompt = f"""
    Basado en el siguiente artículo con título "{title}", realiza dos tareas y devuelve el resultado en formato JSON:
    Formato de salida esperado:
    {{"resumen": "...","etiquetas": ["...", "...", "..."]}}
    1. Genera un "resumen"  conciso en español que no exceda los 200 caracteres.
    2. Genera "etiquetas", una lista de exactamente tres keywords en español.

    Cuerpo del artículo:
    {text}
    """
response = ollama.generate(
            model='llama3.2:latest',
            prompt=prompt,
            format='json',
            stream=False
        )

In [10]:
content = response['response']

In [11]:
data = json.loads(content)
summary = data.get("resumen", "No se pudo generar el resumen.")
tags = data.get("etiquetas", [])

In [12]:
print(data)

{'resumen': 'La porfiria es una enfermedad metabólica rara que puede causar síntomas similares a los de los vampiros, como la fotosensibilidad y la aversión al ajo. La ciencia ha avanzado en su diagnóstico y tratamiento, pero muchos pacientes siguen enfrentando desafíos reales.', 'etiquetas': ['Porfiria', 'Enfermedad metabólica rara', 'Síntomas vampiricos']}


In [13]:
print(summary)

La porfiria es una enfermedad metabólica rara que puede causar síntomas similares a los de los vampiros, como la fotosensibilidad y la aversión al ajo. La ciencia ha avanzado en su diagnóstico y tratamiento, pero muchos pacientes siguen enfrentando desafíos reales.


In [14]:
print(tags)

['Porfiria', 'Enfermedad metabólica rara', 'Síntomas vampiricos']


In [15]:
def chatbot(texto_articulo, resumen_articulo, modelo="llama3.2:latest"):
    
    # Mensaje inicial con el contexto
    messages = [
        {
            "role": "system",
            "content": f"Eres un asistente que responde preguntas sobre el siguiente artículo.\n\nTexto del artículo:\n{texto_articulo}\n\nResumen del artículo:\n{resumen_articulo}\n\nAhora, responde las preguntas del usuario basándote en esta información."
        }
    ]

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break

        messages.append(
            {
                "role": "user",
                "content": user_input,
            }
        )

        response = ollama.chat(
            model=modelo, messages=messages, stream=False
        )

        assistant_response = response["message"]["content"]
        messages.append(
            {
                "role": "assistant",
                "content": assistant_response,
            }
        )

        print(f"Assistant: {assistant_response}")

In [16]:
chatbot(text, summary)


Assistant: Según el artículo, los síntomas de la porfiria pueden variar dependiendo del tipo de enfermedad y su gravedad. Algunos de los síntomas mencionados en el artículo son:

* Fotosensibilidad: los pacientes con porfiria pueden ser muy sensibles a la luz del sol, lo que puede causar picazón o lesiones en la piel.
* Lesiones cutáneas: los pacientes con porfiria eritropoyética heredada pueden sufrir lesiones en la piel que pueden durar varios días después de estar expuestos al sol directo.
* Mutilaciones: algunos pacientes con porfiria pueden sufrir mutilaciones en las orejas y la nariz debido a la acumulación de protoporfirina IX en estas áreas.
* Caída de los dedos: en casos más graves, la porfiria puede causar la caída de los dedos debido a la falta de oxígeno en los tejidos.
* Eritrodoncia: la acumulación de porfirinas en los dientes puede dar lugar a una condición conocida como eritrodoncia, que se caracteriza por un coloración del esmalte en tonos rojizos o marrones.

Es impor

## Ejemplo de uso de un modelo de ollama con la API de OpenAI

In [ ]:
from openai import OpenAI

In [ ]:



client = OpenAI(
    base_url="http://localhost:11434/v1",  # Puerto predeterminado de Ollama
    api_key="ollama", # cadena "dummy" (ficticia), es obligatorio pero no se usa localmente.
)


response = client.chat.completions.create(
    model="llama3.2:latest", # Ej: "llama3", "mistral"
    messages=[
        {"role": "system", "content": "Eres un experto en python."},
        {"role": "user", "content": "¿Qué métodos puedo usar para añadir elementos a una lista de python?"}
    ]
)

print(response.choices[0].message.content)

Excelente pregunta!

En Python, hay varias formas de agregar elementos a una lista. Aquí te presento algunas de las opciones más comunes:

1. **Usando la función `append()`**: esta es una de las formas más sencillas y comunes de agregar elementos a una lista.

    ```python
mis_listas = []
mis_listas.append(10)
print(mis_listas)  # Output: [10]

mi_lista = ['hola']
milista.append('mundo')
print(mi(lista))  # Output: ['hola', 'mundo']
```

2. **Usando la función `extend()`**: esta función permite agregar múltiples elementos a una lista al mismo tiempo.

    ```python
mis_listas = [1, 2, 3]
mis_listas.extend([4, 5])
print(mis_listas)  # Output: [1, 2, 3, 4, 5]

mi_lista = ['hola']
mi_lista.extend(['mundo', 'pi'])
print(mi(lista))  # Output: ['hola', 'mundo', 'pí'], la lista está creciendo
```

3. **Usando el operador `=`**: esta forma no es recomendada, ya que puede crear problemas al intentar acceder a elementos de la lista.

    ```python
mis_listas = [1]
mi_listas[0] = 10
print(mis_li